# Package and Data Import

In [7]:
import sys 
import os 
from pathlib import Path

os.chdir(Path.cwd().parent)
sys.version

'3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) \n[Clang 6.0 (clang-600.0.57)]'

In [1]:
import numpy as np
from src import DataImport
import matplotlib.pyplot as plt
from src import Models
import shap
import pandas as pd 
import xgboost
from sklearn.model_selection import StratifiedKFold, cross_val_score
from src import Evaluate

In [6]:
! pip list

Package                  Version
------------------------ --------
anyio                    3.6.2
appnope                  0.1.3
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
astor                    0.8.1
asttokens                2.2.1
attrs                    22.2.0
autograd                 1.5
autograd-gamma           0.5.0
backcall                 0.2.0
beautifulsoup4           4.11.2
bleach                   6.0.0
cffi                     1.15.1
cloudpickle              2.2.1
comm                     0.1.2
contourpy                1.0.7
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastjsonschema           2.16.2
fonttools                4.38.0
formulaic                0.5.2
fqdn                     1.5.1
future                   0.18.3
graphlib-backport        1.0.3
idna                     3.4
importlib-metadata       6.0.

# dfMort imputed on 22th Feb

## Imputation

In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

df=pd.read_csv(Path.cwd().parent/'Data/HRS/data_preprocess/Data/merge_data_step_3.csv')
df['death'] = [0 if np.isnan(x) else 1 for x in df['deathYear']]
df=df.dropna(thresh=50)
df.reset_index(drop=True,inplace=True)
df_to_missing_forest=df.drop(columns=['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn'])



In [3]:
df['death'].value_counts()

0    9803
1    4087
Name: death, dtype: int64

In [ ]:
# missing forest
df_to_missing_forest=df.drop(columns=['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn'])
imputer = MissForest(n_estimators=500, max_iter=5,criterion='squared_error')
imputed_data = imputer.fit_transform(df_to_missing_forest)


In [13]:
df_imputed = pd.DataFrame(columns=df_to_missing_forest.columns,data=imputed_data)
df_imputed.to_csv(Path.cwd().parent/'Data/HRS/data_preprocess/missingforest/df_by_us_no_missing_computed_20230226_before_merge.csv',index=False)


In [16]:
###### data_non_missing_version_2 = pd.DataFrame(columns=df_to_missing_forest.columns,data=imputed_data)
data_non_missing_version_2=df_imputed.copy()
data_non_missing_version_2.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]=df.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]
data_non_missing_version_2.to_csv(Path.cwd().parent/'Data/HRS/data_preprocess/missingforest/df_by_us_no_missing_computed_20230226_after_merge.csv',index=False)


/var/folders/4x/vv626j3d62g57l8x8_7ksf0r0000gn/T/ipykernel_1738/210735609.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_non_missing_version_2.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]=df.loc[:,['death','deathYear','deathMonth','deathReason','year_check','hhidpn','hhid','pn']]


In [17]:
data_non_missing_version_2['death'].value_counts()

0    9803
1    4087
Name: death, dtype: int64

## binary vars treatment

In [18]:
import pandas as pd
import numpy as np
df=pd.read_csv(Path.cwd().parent/'Data/HRS/data_preprocess/missingforest/df_by_us_no_missing_computed_20230226_after_merge.csv')

binary_columns = [x for x in df.columns if 'YN' in x]
for column in binary_columns:
    print(f'\nbefore: \n{df[column].value_counts()}')
    temp=recode_categorical_vars(column,2,df)
    print(f'after: {df[column].value_counts()}')




before: 
-1.0    8291
 1.0    5599
Name: maleYN, dtype: int64
[[-1.0, 8291], [1.0, 5599]]
cut_point {-1.0: 0.19380849532037447}
bins [-99.80619150467963, 0.19380849532037447, 100.19380849532037]
after: -1.0    8291
 1.0    5599
Name: maleYN, dtype: int64

before: 
-1.000    12138
 1.000     1751
-0.152        1
Name: blackYN, dtype: int64
[[-1.0, 12138], [1.0, 1751]]
cut_point {-1.0: 0.7478580171358629}
bins [-99.25214198286413, 0.7478580171358629, 100.74785801713587]
after: -1.0    12139
 1.0     1751
Name: blackYN, dtype: int64

before: 
 1.000    12592
-1.000     1284
 0.720        2
 0.544        1
 0.128        1
 0.856        1
 0.740        1
 0.948        1
 0.644        1
 0.872        1
 0.568        1
 0.972        1
-0.148        1
 0.876        1
-0.488        1
Name: migrantYN, dtype: int64
[[1.0, 12592], [-1.0, 1284]]
cut_point {-1.0: -0.8149322571346209}
bins [-100.81493225713461, -0.8149322571346209, 99.18506774286539]
after:  1.0    12606
-1.0     1284
Name: migrantY

In [19]:
income_99_qr=np.percentile(df['ZincomeT'], [99.99 ,25])[0]
df['ZincomeT'] = np.where(df['ZincomeT'] >= income_99_qr, income_99_qr, df['ZincomeT'])
wealth_99_qr=np.percentile(df['ZwealthT'], [99.99 ,25])[0]
df['ZwealthT'] = np.where(df['ZwealthT'] >= wealth_99_qr, wealth_99_qr, df['ZwealthT'])

In [17]:
df.to_csv(Path.cwd().parent/'Data/HRS/model_used_data/df_by_us.csv',index=False)

## Performance check

In [10]:
from src import Models
from src import DataImport
from src import Evaluate
import pandas as pd 
from pathlib import Path
domains = DataImport.domain_dict()

df=pd.read_csv(Path.cwd()/'Data/HRS/model_used_data/df_by_us.csv')


In [55]:
for model_selection in ['xgb','lgb']:
    print(f'\n{model_selection}')
    model = Models.Model_fixed_test_size(data=df, test_size=0.3,domain_list=domains['all'], model=model_selection,train_subset_size=1, order=0, y_colname='death')
    evas = Evaluate.metric(model)
    print(f'imv={evas.imv},\nroc-auc={evas.auc_score},\npr-auc={evas.pr_auc},\nf1={evas.pr_f1},\nefron_r2={evas.efron_rsquare},\nffc_r2={evas.ffc_r2},\nIP={evas.pr_no_skill}')



xgb
imv=0.11578468662965002,
roc-auc=0.7972320353537359,
pr-auc=0.6339765459765514,
f1=0.5588768115942029,
efron_r2=0.2029581429135887,
ffc_r2=0.5348794969908781,
IP=0.292913709760362

lgb
imv=0.16898376486769684,
roc-auc=0.8143639436880615,
pr-auc=0.6546368192665374,
f1=0.5446848541862652,
efron_r2=0.2608691927482608,
ffc_r2=0.5686739789110025,
IP=0.292913709760362


# Bio Domain Data Imputation

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from src import DataImport
import re
import sys
import random

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

# ------------------------------------------------------------------------------------------------
# Step1: read data
# ------------------------------------------------------------------------------------------------
bio_08 = pd.read_stata('/Users/valler/OneDrive - Nexus365/Dissertation/biomarker/BIOMK08BL_R.dta')
bio_06 = pd.read_stata('/Users/valler/OneDrive - Nexus365/Dissertation/biomarker/BIOMK06BL_R.dta')

df = DataImport.data_reader_by_us(bio=False)

# Step1 End Here ---------------------------------------------------------------------------------



In [ ]:

# ------------------------------------------------------------------------------------------------
# Step2: combine HHID and PN together for df, bio_06 and bio_08
# ------------------------------------------------------------------------------------------------
df['hhidpn']=df['hhidpn'].astype(int)
df['hhid']=df['hhid'].astype(int)

bio_06['HHID'] = bio_06['HHID'].astype(int)
bio_06['PN'] = bio_06['PN'].astype(int)

bio_08['HHID'] = bio_08['HHID'].astype(int)
bio_08['PN'] = bio_08['PN'].astype(int)

bio_06['hhidpn']=[str(bio_06.loc[index,'HHID'])+'0'+str(bio_06.loc[index,'PN']) for index in bio_06.index]
bio_08['hhidpn']=[str(bio_08.loc[index,'HHID'])+'0'+str(bio_08.loc[index,'PN']) for index in bio_08.index]

bio_06['hhidpn'] = bio_06['hhidpn'].astype(int)
bio_08['hhidpn'] = bio_08['hhidpn'].astype(int)
# Step2 End Here ---------------------------------------------------------------------------------


In [ ]:
# ------------------------------------------------------------------------------------------------
# Step3: preprocess bio_06 and bio_08
# ------------------------------------------------------------------------------------------------
# For bio_06, only keep the following columns
bio_06_columns = ['hhidpn', 'KA1CBIOS', 'KA1C_ADJ',
                  'KHDlBIOS', 'KHDl_ADJ',
                  'KTCBIOS', 'KTC_ADJ',
                  'KCYSC_IMP', 'KCYSC_ADJ',
                  'KCRP_IMP', 'KCRP_ADJ',
                  'KBlVERSION', 'KBIOWGTR']

bio_06=bio_06[bio_06_columns]

bio_06_columns = [x.replace('K','').replace('BIOS','').replace('l','L') for x in bio_06.columns]
bio_06_col_dict = {x:y for x,y in zip(list(bio_06.columns),bio_06_columns)}
bio_06=bio_06.rename(columns=bio_06_col_dict)



# construct 3 key variables for bio_08 (non-adjusted, multiple data sources)
# 1. TC, missing = 299
bio_08['TC'] = [x if np.isnan(y) else y for x,y in zip(bio_08['LTCUW'],bio_08['LTCBIOS'])]
bio_08['TC'].isna().sum()

# 2. HDL , missing = 776
bio_08['HDL'] = [x if np.isnan(y) else y for x,y in zip(bio_08['LHDLBIOS'],bio_08['LHDLUW'])]
bio_08['HDL'].isna().sum()

# 3. HbA1c , missing = 73
bio_08['A1C'] = [x if np.isnan(y) else y for x,y in zip(bio_08['LA1CBIOS'],bio_08['LA1CFLEX'])]
bio_08['A1C'].isna().sum()

bio_08_columns = ['hhidpn','A1C', 'LA1C_ADJ',
                  'HDL', 'LHDL_ADJ',
                  'TC','LTC_ADJ',
                  'LCYSC_IMP','LCYSC_ADJ',
                  'LCRP_IMP','LCRP_ADJ',
                  'LBLVERSION', 'LBIOWGTR']

bio_08 = bio_08[bio_08_columns]
bio_08_columns = [x[1:] if x.startswith('L') else x for x in bio_08.columns]
bio_08_col_dict = {x:y for x,y in zip(list(bio_08.columns),bio_08_columns)}
bio_08=bio_08.rename(columns=bio_08_col_dict)
#bio_08.columns

bio = pd.concat([bio_08,bio_06],axis=0)
# bio.to_csv(Path.cwd()/'model_used_data/bio_all.csv')
#columns: 'A1C', 'A1C_ADJ', 'HDL', 'HDL_ADJ', 'TC', 'TC_ADJ',
#'CYSC_IMP', 'CYSC_ADJ', 'CRP_IMP', 'CRP_ADJ', 'BLVERSION', 'BIOWGTR'



In [ ]:
# ------------------------------------------------------------------------------------------------
# Step4: merge bio_06,bio_08 and df to bio_all_raw and check duplicates of hhidpn
# ------------------------------------------------------------------------------------------------

# combine 06 and 08 with df separately
combined_06 = df.merge(bio_06, left_on=['hhidpn'], right_on=['hhidpn'])
combined_08 = df.merge(bio_08, left_on=['hhidpn'], right_on=['hhidpn'])
column_difference = list(set(combined_08.columns)-set(combined_06.columns))
bio_all_raw = pd.concat([combined_08, combined_06], axis=0)


# bio_all_raw.to_csv(Path.cwd()/'model_used_data/bio_all_with_df_raw.csv')

print('bio_all_raw has {} rows and df has {} rows'.format(len(bio_all_raw),len(df)))

unique_hhidpn = list(set(bio_all_raw['hhidpn']))
print("no overlapped hhidpn in the bio_all_raw, since unique_hhidpn = bio_all_raw = {}".format(len(unique_hhidpn)))

# ------------------------------------------------------------------------------------------------
# TODO: reconstruct the sample weight?
# ------------------------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------------------------
# Step5: missing value imputation
# ------------------------------------------------------------------------------------------------


# check missing
missing_columns = []
for column in bio_all_raw.columns:
    missings = bio_all_raw[column].isna().sum()
    if (missings > 10) & ('death' not in column):
        missing_columns.append(column)
        print('for column {}, there are {} missing'.format(column, missings))

# Make an instance and perform the imputation
# first drop death related variables ['death','death_year','deathYR','death_month'],78 columns left
bio_all_raw_columns = [x if 'death' not in x else 0 for x in bio_all_raw.columns]
while 0 in bio_all_raw_columns:
    bio_all_raw_columns.remove(0)

bio_all_raw_columns_no_deaths = bio_all_raw[bio_all_raw_columns]
# bio_all_raw_columns_no_deaths.to_csv(Path.cwd()/'model_used_data/bio_all_raw_columns_no_deaths.csv')
random.seed(2022)

bio_all_raw_columns_no_deaths=pd.read_csv(Path.cwd()/'model_used_data/bio_all_raw_columns_no_deaths.csv',index_col=0)
imputer = MissForest(n_estimators=500, max_iter=5)
bio_all_raw_columns_no_deaths_no_missing = imputer.fit_transform(bio_all_raw_columns_no_deaths)
# missing values are computed at BMRC


# computation method 1: compute
no_missing = pd.DataFrame(data=bio_all_raw_columns_no_deaths_no_missing, columns=bio_all_raw_columns_no_deaths.columns)
# no_missing['index']=no_missing['index'].astype(int)

# computation method 2

no_missing[['death','death_year','deathYR','death_month','deathReason']]=df[['death','death_year','deathYR','death_month','deathReason']]
no_missing.to_csv(Path.cwd()/'model_used_data/df_by_us_bio.csv',index=False)


# Data Comparison - Figs used in the thesis 

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
from src import Models
from src import Evaluate
from src import DataImport
import matplotlib.pyplot as plt


df = DataImport.data_reader(bio=False)
df_bio = DataImport.data_reader(bio=True)
domains = DataImport.domain_dict()
df_by_us = DataImport.data_reader_by_us(bio=False)
df_by_us_bio = DataImport.data_reader_by_us(bio=True)

model_name = 'xgb'
# non-bio
# note that if you want to run the lgb model, you have to change the kernel to nlp
df.drop(columns=['sampWeight'], inplace=True)

model = Models.Model_fixed_test_size(data=df, test_size=0.3, domain_list=domains['all'], model=model_name,train_subset_size=1,order=0, y_colname='death')

evaluate = Evaluate.metric(model=model)

model_2 = Models.Model_fixed_test_size(data=df_by_us, test_size=0.3, domain_list=domains['all'], model=model_name,train_subset_size=1,order=0, y_colname='death')
evaluate_2 = Evaluate.metric(model=model_2)


# bio
model_2 = Models.Model_fixed_test_size(data=df_by_us_bio, test_size=0.3, domain_list=domains['all_bio'], model=model_name,train_subset_size=1,order=0, y_colname='death')
evaluate_2 = Evaluate.metric(model=model_2)

model_3 = Models.Model_fixed_test_size(data=df_by_us_bio, test_size=0.3, domain_list=domains['all'], model=model_name,train_subset_size=1,order=0, y_colname='death')
evaluate_3 = Evaluate.metric(model=model_3)

# exam the columns that we are interested
def compare_column(column,df,df_by_us):

    # df_by_us=dfMort.copy()
    # df_by_us['death']=[1 if (x==1)& (y<=2015) else 0 for x,y in zip(df_by_us['death'],df_by_us['deathYR'])]
    compare=pd.DataFrame(columns=['hhidpn','interview_year',column+'_author',column+'_us','deathYR_author','deathYR_us'])

    for index,row in df.iterrows():
        hhidpn=row['hhidpn']
        if hhidpn in list(df_by_us.hhidpn):
            row_by_us=df_by_us.loc[df_by_us['hhidpn']==hhidpn,]
            value_by_us=row_by_us[column].values[0]
            interview_year = row_by_us['interview_year'].values[0]

            row_dict={'hhidpn':hhidpn,'interview_year': interview_year,column+'_author':row[column],column+'_us':value_by_us,
                      'deathYR_author':row['deathYR'], 'deathYR_us':row_by_us['deathYR'].values[0]}
            compare=compare.append(row_dict,ignore_index=True)


    compare['equal']=[1 if x==y else 0 for x,y in zip(compare[column+'_author'],compare[column+'_us'])]


    print(compare['equal'].sum(),12982-compare['equal'].sum())
    return compare





def characteristic_compare(df_by_us, df_author):
    null_records=pd.DataFrame(columns=['column','dataset_mark','max','min','mean','uniques','null'])

    count=0
    df_by_us.rename(columns={'death':'deathYN','deathYear':'death_year','deathMonth':'death_month','ZwealthT':'Zwealth', 'ZincomeT':'Zincome'},inplace=True)
    for column in df_by_us.columns:
        if column in list(df_author.columns):

            author_column  = df_author.loc[:,column].replace({' ':None,'True':1,'False':0}).astype(float)
            null_records = null_records.append({'column':column, 'dataset_mark':'author', 'max':max(author_column), 'min':min(author_column), 'mean':np.mean(author_column),'uniques':len(author_column.unique()), 'null':author_column.isnull().sum()},
                                               ignore_index=True)

            count+=1
        else:
            continue

        our_column=df_by_us[column]
        our_column = our_column.replace({' ': None, 'True': 1, 'False': 0}).astype(float)
        null_records=null_records.append({'column':column,'dataset_mark':'us','max':max(our_column),'min':min(our_column), 'mean':np.mean(our_column),'uniques':len(our_column.unique()),'null':our_column.isnull().sum()},ignore_index=True)

    return null_records

null_records = characteristic_compare(df_by_us,df)






## Figures-fig1_death_timeline

In [ ]:
color_yellow='#F1A52C'
color_blue='#001C5B'
# ------------------------
# Figure1
# ------------------------
fontsize_ticks = 12
fontsize_labels = 13

df_deaths= pd.DataFrame(df_by_us[['death','age']].value_counts())
df_deaths=df_deaths.reset_index(drop=False)
df_deaths=df_deaths.loc[df_deaths['death']==1,]


df_by_us['deathYR_half']=[x if y<=6 else x+0.5 for x,y in zip(df_by_us['death_year'],df_by_us['death_month'])]
df_deaths_time=pd.DataFrame(df_by_us[['death','deathYR_half','interview_year']].value_counts())
df_deaths_time=df_deaths_time.reset_index(drop=False)
df_deaths_time=df_deaths_time.loc[df_deaths_time['death']==1,]



plt.figure(figsize=(12, 8))
fig = plt.gcf()
fig.subplots_adjust(left=0.08,right=0.95,top=0.95,bottom=0.1)
# row=0
ax1 = plt.subplot2grid((3, 4), (0, 0), colspan=4)
ax1.bar(df_deaths['age'],df_deaths[0],color=color_blue,alpha=0.8)
ax1.set_xlabel('Age')
ax1.set_title('(a) Death Occurence in age')
ax1.set_ylabel('Death Occurrence',size=fontsize_labels)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.tick_params(axis='both', which='major', labelsize=fontsize_ticks)

# row=1,0
ax2 = plt.subplot2grid((3, 4), (1, 0), colspan=2)
temp=df_deaths_time.loc[df_deaths_time['interview_year']==2006,]
temp=temp.sort_values(by='deathYR_half')
temp['cumulative']=[sum(list(temp[0])[0:x]) for x in np.arange(1,len(list(temp[0]))+1,1)]

ax2.plot(temp['deathYR_half'],temp['cumulative']/sum(temp[0]),color=color_yellow,alpha=0.9)
ax2.set_ylim(0,1)
ax2.set_title('(b) Percentage of Death Occurence in Timeline for 2006 Entry')
ax2.set_xlabel('Death Year',size=fontsize_labels)
ax2.set_ylabel('Percentage',size=fontsize_labels)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=fontsize_ticks)
# row=1,1

ax2_1 = plt.subplot2grid((3, 4), (1, 2), colspan=2)
temp=df_deaths_time.loc[df_deaths_time['interview_year']==2006,]
temp=temp.sort_values(by='deathYR_half')

ax2_1.bar(temp['deathYR_half'],temp[0],width=0.3,color=color_blue,alpha=0.8)
ax2_1.set_ylim(0,170)
ax2_1.set_title('(c) Death Occurence in Timeline for 2006 Entry')
ax2_1.set_xlabel('Death Year',size=fontsize_labels)
ax2_1.set_ylabel('Death Occurrence',size=fontsize_labels)
ax2_1.spines['top'].set_visible(False)
ax2_1.spines['right'].set_visible(False)
ax2_1.tick_params(axis='both', which='major', labelsize=fontsize_ticks)

# The last one is spread on 1 column only, on the 4th column of the second line.
ax3 = plt.subplot2grid((3, 4), (2, 0), colspan=2)
temp=df_deaths_time.loc[df_deaths_time['interview_year']==2008,]
temp=temp.sort_values(by='deathYR_half')
temp['cumulative']=[sum(list(temp[0])[0:x]) for x in np.arange(1,len(list(temp[0]))+1,1)]


ax3.set_title('(d) Percentage of Death Occurence in Timeline for 2008 Entry')
ax3.plot(temp['deathYR_half'],temp['cumulative']/sum(temp[0]),color=color_yellow,alpha=0.9)
ax3.set_ylim(0,1)
ax3.set_xlabel('Death Year',size=fontsize_labels)
ax3.set_ylabel('Percentage',size=fontsize_labels)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.tick_params(axis='both', which='major', labelsize=fontsize_ticks)

ax3_1 = plt.subplot2grid((3, 4), (2, 2), colspan=2)
ax3_1.set_ylim(0,170)
ax3_1.set_title('(e) Death Occurence in Timeline for 2008 Entry')
ax3_1.bar(temp['deathYR_half'],temp[0],width=0.3,color=color_blue,alpha=0.8)
ax3_1.set_xlabel('Death Year',size=fontsize_labels)
ax3_1.set_ylabel('Death Occurrence',size=fontsize_labels)
ax3_1.spines['top'].set_visible(False)
ax3_1.spines['right'].set_visible(False)
ax3_1.tick_params(axis='both', which='major', labelsize=fontsize_ticks)


fig.tight_layout()
# Show the graph
# plt.savefig(Path.cwd()/'graphs/fig1_death_timeline.pdf')
plt.show()



##  Figures-fig2_top_10_continuous

In [ ]:


# ------------------------
# Figure 2
# ------------------------

var_dict=DataImport.variable_dict()
top_10_cols_binary=['currsmokeYN','eversmokeYN','alcoholYN','everdivorced','nevermarried','everunemployed']
top_10_cols_continuous=['Zlifesatis','Zrecentfindiff','Znegaffect']

for var in top_10_cols_binary:
    positive_label_us = [1 if x >0 else 0 for x in df_by_us[var]]
    positive_label = [1 if x > 0 else 0 for x in df[var]]

    print(var_dict[var],'&',round(df_by_us[var].mean(),3),'&',round(df[var].mean(),3),'&',round(np.mean(positive_label_us),3),'&',round(np.mean(positive_label),3), '\\\\')


fontsize_ticks = 12
fontsize_labels = 13
fig,axis = plt.subplots(3,3)
plt.rcParams['figure.figsize']=[13,12]
fig.subplots_adjust(left=0.06, bottom=0.08,top=0.95,right=0.99)
count = 0
colors=[color_blue,color_yellow]
for (m,n),subplot in np.ndenumerate(axis):
    var = top_10_cols_continuous[count]
    if n ==0:

        axis[m,n].hist(df[var],bins=[231,29,238][count],color=color_yellow,alpha=0.9)
        axis[m, n].set_xlabel(var_dict[var],size=fontsize_labels)
        axis[m, n].set_ylabel('Counts',size=fontsize_labels)
        if m ==0:
            axis[m,n].set_title('Author Data')

    elif n==1:
        axis[m, n].hist(df_by_us[var], bins=[231,29,238][count],color=color_blue,alpha=0.8)
        axis[m, n].set_xlabel(var_dict[var],size=fontsize_labels)
        axis[m, n].set_ylabel('Counts',size=fontsize_labels)
        if m ==0:
            axis[m,n].set_title('Our Data')
    else:
        box_1 = axis[m, n].boxplot(df[var], positions=[1], labels=['Author Data'],
                           patch_artist=True, boxprops=dict(color=colors[1]), medianprops=dict(color=colors[1]),
                           whiskerprops=dict(color=colors[1]), capprops=dict(color=colors[1]), showfliers=False)
        box_0 = axis[m, n].boxplot(df_by_us[var], positions=[2], labels=['Our Data'],
                           patch_artist=True, boxprops=dict(color=colors[0]), medianprops=dict(color=colors[0]),
                           whiskerprops=dict(color=colors[0]), capprops=dict(color=colors[0]), showfliers=False)

        box = [box_1, box_0]
        # set the face color of boxes, which is the color of box
        for i in [0,1]:
            for patch in box[i]['boxes']:
                patch.set(facecolor=colors[np.abs(i-1)], alpha=0.4)
        axis[m, n].set_ylabel('Feature Value')
        if m ==0:
            axis[m,n].set_title('Box Plot Comparison')
        count += 1
    axis[m, n].spines['top'].set_visible(False)
    axis[m, n].spines['right'].set_visible(False)
    axis[m,n].tick_params(axis='both', which='major', labelsize=fontsize_ticks)

plt.savefig(Path.cwd()/'graphs/top_10_continuous.pdf')

plt.show()

# Functions 

In [4]:
def recode_categorical_vars(column,cat_num,df):
    
    # pair of values and their number of observations 
    cat_val_count_lst=[]
    count=0
    for index,value in df[column].value_counts().sort_values(ascending=False).items():
        count +=1
        if count>=cat_num+1:
            break
        else:
            # print(index,value)
            cat_val_count_lst.append([index,value])
    print(cat_val_count_lst)
    cat_val_count_lst=sorted(cat_val_count_lst,key=lambda x:x[0])
    # cut off calculation 
    cut_point={}
    for index in range(0,cat_num-1):
        cat_diff = cat_val_count_lst[index+1][0]-cat_val_count_lst[index][0]
        gravity = cat_val_count_lst[index][1]/(cat_val_count_lst[index][1]+cat_val_count_lst[index+1][1])
        cut_point[cat_val_count_lst[index][0]] = cat_val_count_lst[index][0]+gravity*cat_diff
    # cut bins 
    cats = list(cut_point.values())
    cats.sort()
    bins=[cats[0]-100]+cats+[cats[len(cats)-1]+100]
    
    print('cut_point',cut_point)
    #print('cat_val_count_lst',cat_val_count_lst)
    #print(cats)
    print('bins',bins)
    df[column]=pd.cut(df[column],bins=bins,labels=[x[0] for x in cat_val_count_lst])
    df[column]=np.asarray(df[column])
    
    return df
# 
